# Diferential Privacy

In [30]:
# Configuration enviroment

- conda create -n pysyft python=3
- conda activate pysyft # some older version of conda require "source activate pysyft" instead.
- conda install jupyter notebook
- pip install syft
- pip install numpy

In [26]:
import torch

num_entries = 3000
db = torch.rand(num_entries) > 0.5
db

tensor([1, 1, 0,  ..., 0, 0, 1], dtype=torch.uint8)

To ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?"

we must construct what we term "parallel databases" -> databases with one entry removed.

## Generate parallel database

In [28]:
import torch

num_entries = 3000

def create_parallel_db(db, num_index):
    return torch.cat((db[0:num_index], db[num_index + 1:len(db)]))

def create_parallel_dbs(db):
    parallel_dbs = list()
    for i in range(len(db)):
        parallel_dbs.append(create_parallel_db(db, i))
    return parallel_dbs

def get_db_parallel_dbs(num_entries):
    db = torch.rand(num_entries) > 0.5
    pdbs = create_parallel_dbs(db)
    return db, pdbs

db, pdbs = get_db_parallel_dbs(3)
db, pdbs

(tensor([1, 1, 1], dtype=torch.uint8),
 [tensor([1, 1], dtype=torch.uint8),
  tensor([1, 1], dtype=torch.uint8),
  tensor([1, 1], dtype=torch.uint8)])

## Sensitivy

In [21]:
## The maximum amount that the query changes when removing an individual from the database

In [29]:
sensitivy = 0

def query(db):
    return db.sum()

db_result = query(db)
for pdb in pdbs:
    pdb_result = query(pdb)
    diferential = torch.abs(pdb_result - db_result)
    if diferential > sensitivy:
        sensitivy = diferential
    
sensitivy

tensor(1)